<a href="https://colab.research.google.com/github/pea-sys/Kaggle_Public/blob/master/M5_Forecasting_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M5 Forecasting - Accuracy(結果:152/5380 上位3%以内のためSilver Medal)

★今回のコンペでソロ参加銀メダル2枚目となりKaggle Expertになりました。  
  機械学習を独学で0から学んで一年半かかりました。
　

##【概要】(※Official和訳)
第5回目となる今回は、世界最大の売上高を誇るウォルマートの売上データを使って、今後28日間の日次売上高を予測します。  
データは、3つの米国（カリフォルニア、テキサス、ウィスコンシン）の店舗をカバーし、アイテムレベル、部門、製品カテゴリ、店舗の詳細が含まれています。  
さらに、価格、プロモーション、曜日、特別イベントなどの説明変数も含まれています。これらを合わせて、このロバストなデータセットを使用することで、予測精度を向上させることができます。

成功すれば、あなたの研究は予測の理論と実践を前進させ続けることになります。  使用される手法は、適切な在庫やサービスレベルの設定など、様々なビジネス分野に応用することができます。財務省は、ビジネス支援とトレーニングを通じて、ツールと知識の普及を支援し、他の人たちがより正確で校正された予測を達成し、無駄を省き、不確実性とそのリスクの意味を理解できるようにしていきたいと考えています。

## 評価指標
このコンペティションでは、加重根平均二乗尺度誤差（RMSSE）を使用しています。  

[メトリック、スケーリング、重み付けについての詳細](https://mofc.unic.ac.cy/m5-competition/)  
※特殊な評価指標。モデルを学習させる際にはRMSEによる売り上げ数予測にしました

## データセット
[こちら](https://www.kaggle.com/c/m5-forecasting-accuracy/data?select=sales_train_validation.csv)からダウンロード可能
※何も考慮せずに訓練用にデータ加工等するとメモリに載せるのが大変なレべル

## スケジュール
正確には覚えていない。
私は3月26日から6月30日まで、ちょこちょこ休みなく取り組み続けていた。

★活動記録  
今回はあまり記録を残していない。    
LBとCVの相関は確保できないことが分かっていたので  
取り合えずTrustCVを言い聞かせ頑張った

## ●やったことメモ  

#### -- Base
*   [こちら](https://www.kaggle.com/kyakovlev/m5-three-shades-of-dark-darker-magic)のカーネルを流用


#### -- FE
*   価格弾力性やsnapの統合  
 ※価格弾力性は在庫切れや店休日などで売上0になる日があるので、
最小値の代わりに平均値を利用

* Discussionを参考に価格の下一桁と下二桁を特徴化
 [ネタ元](https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/145011)

* 敢えて価格が一意のアイテムの価格特徴量をNaNで埋める(平均や分散等) 

* 月や週の特徴量変換  
[ネタ元](https://qiita.com/shimopino/items/4ef78aa589e43f315113)

* snap前後一日を特徴化

* 米国サマータイムの特徴量追加

[未使用]

* 0スパイク期間が共通するアイテムのグループ化(cumsum)
　※これは結局使わなかった

* イベントや祝日前後日の特徴量

* 各店舗の各カテゴリー内での価格偏差値

#### -- Model

* 以下の２つのモデルをアンサンブル  
  1.   2014年以前のデータのweight=0にしたLGBMモデル
  2.   学習データ全てをweight=1にしたLGBMモデル

  全てのアイテムを無条件にアンサンブルしたわけではなく、予測日から遡り数日間売上0になっているアイテムのみアンサンブルした。  
  理由はモデル1は直近の値を重視しすぎて、0が連続した場合に予測値が低くなりすぎる傾向があったため。

#### -- Validation
* validation scoreが信頼しきれなかったので予測結果をプロット図でひたすら見る  
　※すべては見ていないが二万個程度のプロット図は確認した。上記アンサンブルのヒントになった
* 一応効くだろうと思ったFEが効いていたので、全くvalidationを信じていないわけではなかった


#### -- PostProcess
* 実務ではNGだと思うが、1.05乗数した。
  これは月毎の5月から6月にかけての売上比が予測箇所だけ低かったため。LGBMのため、外挿出来ない代りのようなお気持ちで使用した。ちょっと実務だと使えるか怪しいが、係数をかけているノートブックが沢山あったため使用に踏み切った。



# 感想
ASHRAEの雪辱を晴らせたので良かった。概ねやったことの多くは精度に貢献した。
今回は割とメダル取れる自信はあったので、締め切り三日前位にはやることが終わっていた。特徴量を追加してもほとんど良化しなかったが、2014年以下のweightを0にした途端に効き始めて、みるみるバリデーションスコアが下がってき楽しかった。  
不思議なのが、2014年以下のデータをweight=0にした時と、ドロップした時でrmseが大きく違っていた理由が最後まで分からずじまいだった。  
GoogleDriveのダウンロード制限に引っかかって12時間ダウンロードできない制限があることを知れてよかった。  
今回のコンペは時系列+LB機能しない+final sub1つだけで、検証が困難なコンペだった様に思う。  
LGBMのweightについては、明日以降に理解を深めようと思う。

  







In [ ]:
!nvidia-smi

Wed Jul  1 11:15:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM/
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install --precompile


Cloning into 'LightGBM'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 18291 (delta 0), reused 0 (delta 0), pack-reused 18286
Receiving objects: 100% (18291/18291), 12.28 MiB | 28.79 MiB/s, done.
Resolving deltas: 100% (13362/13362), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 27.48 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
/content/LightGBM
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr

In [ ]:
'''
!pip install -U lightgbm 
#バギングに不具合があり学習が遅いとのこと
#https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/148273
'''

'\n!pip install -U lightgbm \n#バギングに不具合があり学習が遅いとのこと\n#https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/148273\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
%cd /content/drive/My\ Drive/m5-stage

/content/drive/My Drive/m5-stage


In [ ]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
########################### ヘルパー
#################################################################################
## シード生成
# ：すべてのプロセスを確定的にするためのシード     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## マルチプロセス実行
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [ ]:
import calendar
from datetime import datetime as dt
def get_nth_week2(year, month, day, firstweekday=0):
    first_dow = calendar.monthrange(year, month)[0] + 1
    if first_dow == 7:
      first_dow = 0
    offset = (first_dow - firstweekday) % 7
    return (day + offset - 1) // 7 + 1
def encode(df, col):
    # この方法だと場合によって最大値が変化するデータでは正確な値は出ない
    # 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df

def original_fea(data):
  sscaler = StandardScaler()
  sscaler.fit(data['tm_d'].values.reshape(-1, 1))           # xの平均と分散を計算
  y = sscaler.transform(data['tm_d'].values.reshape(-1, 1)) # xを変換

  data['tm_d'] = y
  data['tm_d'] = data['tm_d'].astype('float16')
  del y
  '''
  #食品以外はSnap関係ないので落とす
  for s in ['snap_CA','snap_TX','snap_WI']:
    data.loc[(data['cat_id']!='FOODS') & (data[s]==1),s]=0
  '''
  #Snapカラムをまとめる
  data['snap'] = 0
  data.loc[(data['id'].astype(str).str.contains('CA')) & (data['snap_CA']==1),'snap'] = 1
  data.loc[(data['id'].astype(str).str.contains('TX')) & (data['snap_TX']==1),'snap'] = 1
  data.loc[(data['id'].astype(str).str.contains('WI')) & (data['snap_WI']==1),'snap'] = 1
  data['snap'] = data['snap'].astype("int8")
  del data['snap_CA'],data['snap_TX'],data['snap_WI']
  
  #Snap周辺　CV は上がった
  #CA snap 1-10
  #TX 2,4,8,10,14を除いた1-15
  #WI 1,4,7,10,13を除いた1-15
  data.loc[(data['id'].astype(str).str.contains('CA')) & (data['tm_d'].isin([11])),'snap'] = 2
  data.loc[(data['id'].astype(str).str.contains('TX')) & (data['tm_d'].isin([2,4,8,10,14,16])),'snap'] = 2
  data.loc[(data['id'].astype(str).str.contains('WI')) & (data['tm_d'].isin([1,4,7,10,13,16])),'snap'] = 2
  #data['snap'] = data['snap'].astype("category")

  '''
  data['event_name_1'] = data['event_name_1'].astype(str)
  data['event_type_1'] = data['event_type_1'].astype(str)
  data.loc[(data['tm_m'] == 3) & (data['tm_d'] == 31) & (data['id'].astype(str).str.contains('CA')),'event_name_1'] = 'CA Holiday'
  data.loc[(data['tm_m'] == 3) & (data['tm_d'] == 31) & (data['id'].astype(str).str.contains('CA')),'event_type_1'] = 'National'
  data.loc[(data['tm_y'] == 0) & (data['tm_m'] == 11) & (data['tm_d'] == 27),'event_name_1'] = 'Black Friday'
  data.loc[(data['tm_y'] == 0) & (data['tm_m'] == 11) & (data['tm_d'] == 27),'event_type_1'] = 'Cultural'
  data.loc[(data['tm_y'] == 1) & (data['tm_m'] == 11) & (data['tm_d'] == 23),'event_name_1'] = 'Black Friday'
  data.loc[(data['tm_y'] == 1) & (data['tm_m'] == 11) & (data['tm_d'] == 23),'event_type_1'] = 'Cultural'
  data.loc[(data['tm_y'] == 2) & (data['tm_m'] == 11) & (data['tm_d'] == 27),'event_name_1'] = 'Black Friday'
  data.loc[(data['tm_y'] == 2) & (data['tm_m'] == 11) & (data['tm_d'] == 27),'event_type_1'] = 'Cultural'
  data.loc[(data['tm_y'] == 3) & (data['tm_m'] == 11) & (data['tm_d'] == 28),'event_name_1'] = 'Black Friday'
  data.loc[(data['tm_y'] == 3) & (data['tm_m'] == 11) & (data['tm_d'] == 28),'event_type_1'] = 'Cultural'
  data.loc[(data['tm_y'] == 4) & (data['tm_m'] == 11) & (data['tm_d'] == 27),'event_name_1'] = 'Black Friday'
  data.loc[(data['tm_y'] == 4) & (data['tm_m'] == 11) & (data['tm_d'] == 27),'event_type_1'] = 'Cultural'
  data['event_name_1'] = data['event_name_1'].astype("category")
  data['event_type_1'] = data['event_type_1'].astype("category")
  '''
  '''
  data['event_name_1'] = data['event_name_1'].astype(str)
  bfr = data[~data['event_name_1'].isnull()]['d'].unique() - 1
  aft = data[~data['event_name_1'].isnull()]['d'].unique() + 1
  data.loc[(data['d'].isin(bfr)) & (data['event_name_1'].isnull()),'event_name_1'] = 'Before'
  data.loc[(data['d'].isin(aft)) & (data['event_name_1'].isnull()),'event_name_1'] = 'After'
  data['event_name_1'] = data['event_name_1'].astype("category")

  data.loc[(data['tm_m'] == 3) & (data['tm_d'] == 31) & (data['id'].astype(str).str.contains('CA')),"wday"] = 0
  data.loc[(data['tm_y'] == 0) & (data['tm_m'] == 11) & (data['tm_d'] == 27),"wday"] = 0
  data.loc[(data['tm_y'] == 1) & (data['tm_m'] == 11) & (data['tm_d'] == 23),"wday"] = 0
  data.loc[(data['tm_y'] == 2) & (data['tm_m'] == 11) & (data['tm_d'] == 27),"wday"] = 0
  data.loc[(data['tm_y'] == 3) & (data['tm_m'] == 11) & (data['tm_d'] == 28),"wday"] = 0
  data.loc[(data['tm_y'] == 4) & (data['tm_m'] == 11) & (data['tm_d'] == 27),"wday"] = 0
  data.loc[~data['event_name_1'].isnull(),"wday"] = 0
  '''
  
  data['price_sign'] = -1
  data['price_sign_last'] = -1
  #data = data.loc[~data['sell_price'].isnull()]
  data.loc[~data['sell_price'].isnull(),'price_sign'] = (data.loc[~data['sell_price'].isnull()]['sell_price'] * 100 % 100).astype("int8")
  data.loc[~data['sell_price'].isnull(),'price_sign_last'] =  data.loc[~data['sell_price'].isnull()]['price_sign'].astype(str).str[-1].astype("int8")

  data = encode(data, 'tm_dw')
  data = encode(data, 'tm_d')
  data = encode(data, 'tm_m')
  
  #weekの計算方法を年初ベースに変更
  '''
  data['date'] = (2011 + data['tm_y']).astype(str) +'年'+ data['tm_m'].astype(str)+'月' + data['tm_d'].astype(str)+'日0時0分'
  data['date'] = pd.to_datetime(data['date'], format='%Y年%m月%d日%H時%M分')
  data['tm_w'] = data['date'].apply(lambda x: get_nth_week2(x.year,x.month,x.da y))
  data['tm_w'] = data['tm_w'].astype('int8')
  del data['date']
  '''

  data['price_position'] = 0
  data['price_position'] = data['price_position'].astype('int8')
  data.loc[data['sell_price'] == data['price_mean'], 'price_position'] = 0
  data.loc[data['sell_price'] < data['price_mean'], 'price_position'] = -1
  data.loc[data['sell_price'] > data['price_mean'], 'price_position'] = 1
  data['price_position'] =data['price_position'].astype("int8")



  data = item_group(data)
  data = elasticity(data)
  #data = add_itemcount(data)
  data = add_summertime(data)
  data = fill_static_price(data)
  data =  add_deviation_value(data)
  return data

import csv
def elasticity(data):
  
  #売値固定情報量削減 
  tmp = data[['item_id','sales']].groupby(['item_id']).max().reset_index().sort_values('item_id')
  tmp['demand_mean'] = data[['item_id','sales']].groupby(['item_id']).mean().reset_index().sort_values('item_id')['sales']
  tmp['demand_gap'] =  tmp['sales'] / tmp['demand_mean']
  del tmp['sales'],tmp['demand_mean']
  data = data.merge(tmp,on='item_id',how='left')
  data['Elasticity'] = (data['price_max'] - data['price_mean']) / data['demand_gap']
  del data['demand_gap']
  return data

def item_group(data):
  '''
  with open('sample_writer_row.csv') as f:
    reader = csv.reader(f)
    l = [row for row in reader]

  for i in range(0,len(l)):
    data['gap_group'] = 0
    data.loc[data['item_id'].isin(l[i]),'gap_group'] = i
  data['gap_group'] = data['gap_group'].astype('int8')
  '''
  return data

def add_itemcount(data):
  tmp = data.groupby(['d','dept_id']).count().reset_index()
  tmp = tmp.rename(columns={'id':'item_count'})
  tmp = tmp[['d','dept_id','item_count']]
  return data.merge(tmp,on=['d','dept_id'],how='left')

def add_summertime(data):
  data['IsSummerTime'] = False
  data.loc[((data['tm_m'] >= 4) & (data['tm_m'] <= 10)),'IsSummerTime'] = True
  data.loc[((data['tm_y']==0) & (data['tm_m']==3) & (data['tm_d']>=13)) | ((data['tm_y']==0) & (data['tm_m']==11) & (data['tm_d']<=6)),'IsSummerTime'] = True
  data.loc[((data['tm_y']==1) & (data['tm_m']==3) & (data['tm_d']>=11)) | ((data['tm_y']==1) & (data['tm_m']==11) & (data['tm_d']<=4)),'IsSummerTime'] = True
  data.loc[((data['tm_y']==2) & (data['tm_m']==3) & (data['tm_d']>=10)) | ((data['tm_y']==2) & (data['tm_m']==11) & (data['tm_d']<=3)),'IsSummerTime'] = True
  data.loc[((data['tm_y']==3) & (data['tm_m']==3) & (data['tm_d']>=9)) | ((data['tm_y']==3) & (data['tm_m']==11) & (data['tm_d']<=2)),'IsSummerTime'] = True
  data.loc[((data['tm_y']==4) & (data['tm_m']==3) & (data['tm_d']>=8)) | ((data['tm_y']==4) & (data['tm_m']==11) & (data['tm_d']<=1)),'IsSummerTime'] = True
  data.loc[((data['tm_y']==5) & (data['tm_m']==3) & (data['tm_d']>=13)) | ((data['tm_y']==5) & (data['tm_m']==11) & (data['tm_d']<=6)),'IsSummerTime'] = True
  return data

def fill_static_price(data):
  nancol = ['price_std','price_norm','price_mean','price_nunique','price_momentum','price_momentum_m','price_momentum_y']
  for c in nancol:
    data.loc[data['price_max'] == data['price_min'], c] = np.nan
  data.loc[data['price_std'].isnull(),'price_max'] = np.nan
  data.loc[data['price_std'].isnull(),'price_min'] = np.nan
  return data

def add_deviation_value(data):
  '''
  price_mean = data[['d','cat_id','sell_price']].groupby(['d','cat_id']).mean().reset_index()
  price_mean.rename(columns={'sell_price':'sell_price_allmean'},inplace=True)
  data = data.merge(price_mean,on=['d','cat_id'],how='left')
  data['sell_price_allstd'] = (data['sell_price'] - data['sell_price_allmean']) ** 2
  price_mean = data[['d','cat_id','sell_price_allstd']].groupby(['d','cat_id']).sum().reset_index()

  price_mean['sell_price_allstd'] = np.sqrt(price_mean['sell_price_allstd'] / data[['d','cat_id','sell_price_allmean']].groupby(['d','cat_id']).count().reset_index()['sell_price_allmean'])
  data = data.merge(price_mean,on=['d','cat_id'],how='left')
  data['sell_price_allstd'] = ((data['sell_price'] - data['sell_price_allmean']) / data['sell_price_allstd_y']) #* 10 + 50
  del data['sell_price_allstd_x'],data['sell_price_allstd_y'],data['sell_price_allmean']
  '''
  return data

In [ ]:
def set_weight(data):
  data['weight'] = 1
  data.loc[(data['tm_y']<=4),'weight'] = 0

  #tmp = data.groupby('id').mean().reset_index()[['id','sales']]
  #tmp = tmp.loc[(~tmp['sales'].isnull()) & (tmp['sales']>2)][['id']]
  #data.loc[(data['weight']==0) & (data['id'].isin(tmp['id'])),'weight'] = 0.3

  #data.loc[(data['weight']==0) & (data['tm_m']>=4) & (data['tm_m']<=6),'weight'] = 0.2
  return data

In [ ]:
########################### store IDによってデータをロードするヘルパー
#################################################################################
# データ読込
def get_data_by_store(store):
    
    # 基本特徴量を読んで連絡する
    # BASE,PRICE,CALENDARは「【日本語】M5 - Simple FE」を参照
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # 関連する店舗のみを残す
    df = df[df['store_id']==store]
    
    # メモリの制限があるため、LAGとエンコード特徴量を個別に読み取り、
    # 不要なアイテムを削除する必要があります。 
    # 特徴量グリッドが整列されるため、必要な行のみを保持するために index を使用できます
    # concatはmergeよりも少ないメモリを使用するため、整列は適切です。
    df2 = pd.read_pickle(MEAN_ENC)[mean_features] # 「M5 - Custom features」を参照
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:] # 「M5 - Custom features」を参照
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # メモリ制限に達しないように削除
    
    df = pd.concat([df, df3], axis=1)
    del df3 # メモリ制限に達しないように削除
    df = original_fea(df)
    
    # 特徴量リストを作成する
    features = [col for col in list(df) if col not in remove_features]
    print(features)
    df = df[['id','d',TARGET]+features]
    df = set_weight(df)
    # 最初のn行をスキップ
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# トレーニング後にテストを再結合
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### 動的移動LAGを作成するヘルパー
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [ ]:
########################### モデルパラメータ
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

In [ ]:
########################### 変数
#################################################################################
VER = 1                          # バージョン
SEED = 42                        # 
seed_everything(SEED)            # 私たちはすべてのものが出来るだけ
lgb_params['seed'] = SEED        # 確定的であることを望みます。
N_CORES = psutil.cpu_count()     # 使用可能なCPUコア


#限界と定数
TARGET      = 'sales'            # ターゲット
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1941               # trainの終了日
P_HORIZON   = 28                 # 予測期間
USE_AUX     = False               # 事前学習済みモデルを使用するかどうか

#削除する特徴量
## これらの機能はオーバーフィットにつながります 
## または test に存在しない値
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 


#特徴量のパス
ORIGINAL = ''
BASE     = 'grid_part_1.pkl'#../input/m5-simple-fe/
PRICE    = 'grid_part_2.pkl'#../input/m5-simple-fe/
CALENDAR = 'grid_part_3.pkl'#../input/m5-simple-fe/
LAGS     = 'lags_df_28.pkl'#../input/m5-lags-features/
MEAN_ENC = 'mean_encoding_df.pkl'#../input/m5-custom-features/




#STORES id
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_evaluation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
STORES_IDS = ['CA_3','TX_3','WI_2']
#STORES_IDS = ['CA_1','CA_2','CA_4']

#LAG作成用の分割
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,21,30,60]:#実験21
        ROLS_SPLIT.append([i,j])

In [ ]:
grid_df, features_columns = get_data_by_store('TX_1')
grid_df.sample()

['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7

,id,d,sales,item_id,dept_id,cat_id,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_item_id_mean,enc_item_id_std,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,...,sales_lag_38,sales_lag_39,sales_lag_40,sales_lag_41,sales_lag_42,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60,rolling_mean_180,rolling_std_180,rolling_mean_tmp_1_7,rolling_mean_tmp_1_14,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60,snap,price_sign,price_sign_last,tm_dw_cos,tm_dw_sin,tm_d_cos,tm_d_sin,tm_m_cos,tm_m_sin,price_position,Elasticity,IsSummerTime,weight
967271,FOODS_3_694_TX_1_evaluation,550,25.0,FOODS_3_694,FOODS_3,FOODS,0,1.580078,1.679688,1.480469,0.076416,1.605469,0.94043,3.0,32,1.0,0.991211,1.007812,NaN,NaN,NaN,NaN,1.740234,31,7,1,5,1,0,2.109375,5.769531,2.626953,7.058594,20.375,13.3125,38.0,22.0,32.0,24.0,25.0,...,44.0,34.0,23.0,27.0,18.0,26.859375,5.898438,29.140625,8.859375,29.296875,8.085938,27.59375,8.164062,22.296875,8.460938,25.28125,24.640625,26.171875,27.984375,24.71875,23.921875,25.703125,27.828125,23.140625,25.71875,27.296875,27.78125,0,58,8,0.5,0.866025,1.0,-0.001935,-0.866025,-0.5,-1,0.021565,True,0


In [ ]:
'''
def drop_outliner(df,store_id):
  print(df.shape)
  if store_id == 'TX_1':
    df = df[~((grid_df['item_id']=='HOUSEHOLD_2_466') & (df['sell_price'] >6.968750))]
  elif store_id == 'WI_2':
    df = df.loc[~((df['item_id']=='HOUSEHOLD_2_406') & (df['tm_y']==1))]
  
  df = df.loc[~((df['item_id']=='HOUSEHOLD_2_292') & (df['tm_y']<3))]
  df = df.loc[~((df['item_id']=='HOUSEHOLD_1_259') & (df['tm_y']<4))]
  print(df.shape)
  return df
'''

"\ndef drop_outliner(df,store_id):\n  print(df.shape)\n  if store_id == 'TX_1':\n    df = df[~((grid_df['item_id']=='HOUSEHOLD_2_466') & (df['sell_price'] >6.968750))]\n  elif store_id == 'WI_2':\n    df = df.loc[~((df['item_id']=='HOUSEHOLD_2_406') & (df['tm_y']==1))]\n  \n  df = df.loc[~((df['item_id']=='HOUSEHOLD_2_292') & (df['tm_y']<3))]\n  df = df.loc[~((df['item_id']=='HOUSEHOLD_1_259') & (df['tm_y']<4))]\n  print(df.shape)\n  return df\n"

In [ ]:
########################### AUXモデル

if USE_AUX:
    lgb_params['n_estimators'] = 2
    
''' 
Train CA_3
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'snap', 'price_sign', 'price_sign_last', 'Elasticity', 'IsSummerTime']
[100]	valid_0's rmse: 2.28421
[200]	valid_0's rmse: 2.13191
[300]	valid_0's rmse: 2.04963
[400]	valid_0's rmse: 1.97499
[500]	valid_0's rmse: 1.9129
[600]	valid_0's rmse: 1.8525
[700]	valid_0's rmse: 1.7945
[800]	valid_0's rmse: 1.73828
[900]	valid_0's rmse: 1.68724
[1000]	valid_0's rmse: 1.6407
[1100]	valid_0's rmse: 1.59522
[1200]	valid_0's rmse: 1.5517
[1300]	valid_0's rmse: 1.51027
[1400]	valid_0's rmse: 1.47179
rm: cannot remove 'train_data.bin': No such file or directory
Train TX_3
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'snap', 'price_sign', 'price_sign_last', 'Elasticity', 'IsSummerTime']
[100]	valid_0's rmse: 1.77015
[200]	valid_0's rmse: 1.63885
[300]	valid_0's rmse: 1.56974
[400]	valid_0's rmse: 1.51605
[500]	valid_0's rmse: 1.46463
[600]	valid_0's rmse: 1.41385
[700]	valid_0's rmse: 1.36775
[800]	valid_0's rmse: 1.32441
[900]	valid_0's rmse: 1.28204
[1000]	valid_0's rmse: 1.2446
[1100]	valid_0's rmse: 1.20861
[1200]	valid_0's rmse: 1.17677
[1300]	valid_0's rmse: 1.14591
[1400]	valid_0's rmse: 1.11877
rm: cannot remove 'train_data.bin': No such file or directory
Train WI_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'snap', 'price_sign', 'price_sign_last', 'Elasticity', 'IsSummerTime']
[100]	valid_0's rmse: 2.56866
[200]	valid_0's rmse: 2.32451
[300]	valid_0's rmse: 2.18298
[400]	valid_0's rmse: 2.06812
[500]	valid_0's rmse: 1.96433
[600]	valid_0's rmse: 1.87574
[700]	valid_0's rmse: 1.80038
[800]	valid_0's rmse: 1.73619
[900]	valid_0's rmse: 1.67442
[1000]	valid_0's rmse: 1.621
[1100]	valid_0's rmse: 1.57173
[1200]	valid_0's rmse: 1.52515
[1300]	valid_0's rmse: 1.48612
[1400]	valid_0's rmse: 1.44579
rm: cannot remove 'train_data.bin': No such file or directory
'''

" \nTrain CA_3\n['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'ro

In [ ]:
########################### モデルのトレーニング
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # 現在の店舗のグリッドを取得
    grid_df, features_columns = get_data_by_store(store_id)
    #grid_df = drop_outliner(grid_df,store_id)
    # マスク
    # Train （1913未満のすべてのデータ）
    # "Validation" （過去28日間-実際の検証セットではない
    # Test （1913日を超えるすべてのデータ、再帰的機能のギャップあり）
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    # マスクを適用してlgbデータセットをbinとして保存し、
    # dtype変換中のメモリスパイクを削減する
    # https://github.com/Microsoft/LightGBM/issues/1032
    # 変換を回避するには、常にnp.float32を使用するか、
    # トレーニングを開始する前に bin に保存する必要があります
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                        weight=grid_df[train_mask]['weight'],
                       label=grid_df[train_mask][TARGET])#                  
    train_data.free_raw_data = None
    #train_data.save_binary('train_data.bin')
    #train_data = lgb.Dataset('train_data.bin')
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                              weight=grid_df[valid_mask]['weight'],
                       label=grid_df[valid_mask][TARGET])#     ,                                     
    valid_data.free_raw_data = None
    # 後の予測のためにデータセットの一部を保存する
    # 再帰的に計算する必要がある機能を削除する
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    # シーダーを再度起動して、
    # 各「コード行」np.randomが「進化」するlgbトレーニングを100％確定的にするため、
    # 「リセット」する必要がある場合があります。
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    # モデルを保存-実際の「.bin」ではなく
    # estimator = lgb.Booster(model_file='model.txt')
    # pickleファイルとすると最良の反復（または保存反復）でのみ予測できます
    # pickle.dumpは柔軟性を提供します。
    # estimator.predict(TEST, num_iteration=100)
    # num_iteration - 予測したい反復回数
    # NULLまたは<= 0は最適な反復を使用することを意味します
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))
    # 一時ファイルとオブジェクトを削除して、
    # HDDスペースとRAMメモリを解放します
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # 予測のためのモデル特徴量の保持
    MODEL_FEATURES = features_columns

In [ ]:
########################### 予測
#################################################################################

# 予測を保存するためのダミーデータフレームを作成する
all_preds = pd.DataFrame()

# テストデータセットをトレーニングデータの一部に結合して、
# 再帰的な特徴量を作成します
base_test = get_base_test()

# 予測時間を測定するタイマー
main_time = time.time()

# 各予測日にループする移動LAGは最も時間がかかるため、
# 1日全体で計算します。
for PREDICT_DAY in range(1,P_HORIZON+1):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # 移動LAGを計算するための一時的なグリッドを作成する
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # すべてのモデルを読んで、日/店舗のペアごとに予測を行う
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
 
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # 適切な列の名前を付け、all_preds に追加します
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  3.01 min round |  3.01 min total |  39759.87 day sales |
Predict | Day: 2
##########  2.90 min round |  5.91 min total |  37064.22 day sales |
Predict | Day: 3
##########  3.02 min round |  8.93 min total |  37035.72 day sales |
Predict | Day: 4
##########  3.03 min round |  11.96 min total |  36843.94 day sales |
Predict | Day: 5
##########  3.05 min round |  15.01 min total |  41757.13 day sales |
Predict | Day: 6
##########  3.05 min round |  18.06 min total |  49965.75 day sales |
Predict | Day: 7
##########  3.06 min round |  21.12 min total |  50753.83 day sales |
Predict | Day: 8
##########  3.06 min round |  24.18 min total |  45454.29 day sales |
Predict | Day: 9
##########  3.00 min round |  27.18 min total |  39027.52 day sales |
Predict | Day: 10
##########  2.93 min round |  30.11 min total |  44304.91 day sales |
Predict | Day: 11
##########  2.99 min round |  33.10 min total |  45306.22 day sales |
Predict | Day: 12
##########  3.05 min round

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.896919,0.808327,0.787733,0.800273,1.044852,1.353330,1.283072,0.971777,0.788433,1.030200,0.827920,1.163072,1.399896,1.219377,0.939904,0.912041,0.868280,0.828838,0.973320,1.238219,1.288867,1.000805,0.921084,0.926695,0.971500,1.155859,1.203079,1.210900
1,HOBBIES_1_002_CA_1_evaluation,0.220466,0.204749,0.212473,0.199276,0.220701,0.305867,0.324968,0.213049,0.194676,0.211614,0.209575,0.319597,0.379714,0.297997,0.204043,0.233662,0.214412,0.237179,0.274404,0.329860,0.343886,0.211545,0.210241,0.205603,0.232045,0.266742,0.314893,0.399803
2,HOBBIES_1_003_CA_1_evaluation,0.529796,0.480658,0.485227,0.512719,0.711414,0.794804,0.726323,0.563683,0.519169,0.558241,0.450290,0.648836,0.777947,0.667974,0.498328,0.485776,0.490877,0.475087,0.553368,0.739833,0.701520,0.499819,0.445005,0.473900,0.483154,0.658015,0.758736,0.704474
3,HOBBIES_1_004_CA_1_evaluation,1.679303,1.350790,1.390648,1.472831,1.742545,2.423991,2.897262,2.091391,1.362763,1.384726,1.576862,2.081480,2.863150,2.804167,1.817179,1.660786,1.362049,1.440221,1.874175,2.524346,3.068022,1.936250,1.373735,1.419908,1.443614,1.886773,2.622977,3.134182
4,HOBBIES_1_005_CA_1_evaluation,1.015119,0.977014,0.927098,0.993035,1.080928,1.303042,1.378404,1.250934,1.054353,1.132265,1.105215,1.377746,1.400710,1.540751,1.097919,1.049561,0.998681,1.012499,1.142446,1.346198,1.401818,1.146235,0.986988,1.073486,1.128597,1.389739,1.579713,1.455819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.476450,0.443781,0.437969,0.483285,0.471879,0.511311,0.668452,0.499746,0.490234,0.436963,0.646053,0.730032,0.706655,0.783020,0.620229,0.549439,0.520738,0.592746,0.562463,0.711263,0.877204,0.616886,0.582591,0.584170,0.486450,0.551508,0.578857,0.728974
30486,FOODS_3_824_WI_3_evaluation,0.250087,0.239587,0.220720,0.192613,0.209839,0.281332,0.285680,0.277420,0.243141,0.226478,0.326821,0.315657,0.284630,0.318896,0.280560,0.296365,0.275089,0.292407,0.244004,0.300197,0.386606,0.287312,0.311400,0.287994,0.239931,0.208813,0.264203,0.345811
30487,FOODS_3_825_WI_3_evaluation,0.663887,0.480266,0.514194,0.494664,0.553251,0.645590,0.727649,0.728169,0.494397,0.652815,0.993464,1.103823,0.854273,1.083670,1.077858,0.775913,0.887848,0.875240,0.843881,1.073937,1.255404,0.937257,0.987630,0.981570,0.670349,0.709838,0.738259,0.931900
30488,FOODS_3_826_WI_3_evaluation,1.018613,1.053016,0.957609,0.994166,1.000394,1.178593,1.157526,1.221021,1.052750,1.008126,1.313052,1.604317,1.451344,1.762432,1.243546,1.156422,1.183751,1.257486,1.169555,1.537536,1.723522,1.168916,1.462073,1.250036,1.044753,1.122169,1.159464,1.296833


In [ ]:
VER

1

In [ ]:
########################### 書き出す
#################################################################################
# コンぺのサンプル提出を読んで、予測を結合する
# 「_validation」データのみの予測があるため
# 「_evaluation」アイテムに対してfillna（）を実行する必要がある
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False)